In [1]:
import pandas as pd
from bnlp.corpus import stopwords, punctuations, digits
from bnlp import BasicTokenizer
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

C:\Users\meera\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
threshold = .7

In [3]:
train_set = pd.read_csv("train.csv")

In [4]:
print(train_set)
print(len(train_set))

                                                   text  label    class
0     মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...      1  sadness
1     অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে  কেননা আমার...      1  sadness
2     আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...      3  disgust
3     মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...      1  sadness
4     এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...      4     fear
...                                                 ...    ...      ...
4399  আর  এত  টাকার  কাজই   বা  কিভাবে   করলো আজ  কা...      3  disgust
4400  আমি মনে করি হাফপ্যান্ট পরে নাচগান হচ্ছে নোংরাম...      3  disgust
4401  বিরক্ত করাও সম্ভবত এক ধরণের ভালোবাসা এই ইগোয়িস...      1  sadness
4402  এই মূহুর্তে আমার আর সহ্য হচ্ছে না সমস্যাগুলো ম...      3  disgust
4403  ২২ শ্রাবণের মতো থ্রিলার বানানো সৃজিত এমন অসংলগ...      3  disgust

[4404 rows x 3 columns]
4404


In [5]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [6]:
def filters(corpus):
    res = []
    for i in corpus:
        temp_i = [*i]
        flag = 0
        for j in stopwords:
#             print(f"i={temp_i} j={[*j]} jac={jaccard(temp_i,[*j])}")
            if jaccard(temp_i,j)>threshold:
                flag = 1
                break
        if(flag == 1):
            continue
            
        if i in stopwords:
            continue

        if i in punctuations + '‘' + '’':
            continue
        if i in digits:
            continue

        res.append(i)

    return res

In [7]:
tokenizer = BasicTokenizer()
tokenizer_bert = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

In [8]:
train_set_text_list = train_set.text.tolist()
train_set_label_list = train_set.label.tolist()
print(len(train_set_text_list))
print(len(train_set_label_list))

4404
4404


In [9]:
new_train_set_text_list = []
for i in train_set_text_list:
    temp_list = tokenizer.tokenize(i)
    new_train_set_text_list.append(filters(temp_list))
# print(new_train_set_text_list)
    

In [10]:
print(len(new_train_set_text_list))

4404


In [11]:
final_list = []
for i in new_train_set_text_list:
    final_list.append(" ".join(i))
# print(final_list)

In [12]:
for i in range(len(final_list)):
    if type(final_list[i])!=str:
        print(i)
    if len(final_list[i])==0:
        print(i)

1800


In [13]:
for i in range(len(final_list)):
    if type(final_list[i])!=str:
        print(i)
    if len(final_list[i])==0:
        final_list[i]=train_set_text_list[i]

In [14]:
for i in range(len(final_list)):
    if type(final_list[i])!=str:
        print(i)
    if len(final_list[i])==0:
        print(i)

In [15]:
write_csv = pd.DataFrame(list(zip(final_list, train_set_label_list)),
               columns =['text', 'label'])

In [16]:
write_csv.to_csv("jaccard_train.csv",index=False)